## Requirements

* set the directories in the next cell

### Installing Julia and ABCD

We use the command line interface option to run ABCD below. 
The following steps are required:

* install Julia (we used version 1.4.2) from https://julialang.org/downloads/
* download ABCD from https://github.com/bkamins/ABCDGraphGenerator.jl
* adjust the 'abcd_path' in the next cell to the location of the 'utils' subdirectory of ABCD
* run 'julia abcd_path/install.jl' to install the required packages


In [ ]:
## set those accordingly
datadir = '../Datasets/'
abcd_path = '~/ABCD/utils/'

In [ ]:
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from collections import Counter
import os
import umap
import pickle
import partition_igraph
import subprocess
from sklearn.metrics import adjusted_mutual_info_score as AMI

## we use those for the book, but you can change to other colors
cls_edges = 'gainsboro'
cls = ['silver','dimgray','black']

## Zachary (karate) graph

A small graph with 34 nodes and two "ground-truth" communities;
modularity-based algorithms will typically find 4 or 5 communities.

In [ ]:
z = ig.Graph.Famous('zachary')
z.vs['size'] = 12
z.vs['name'] = [str(i) for i in range(z.vcount())]
z.vs['label'] = [str(i) for i in range(z.vcount())]
z.vs['label_size'] = 8
z.es['color'] = cls_edges
z.vs['comm'] = [0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1]
z.vs['color'] = [cls[i] for i in z.vs['comm']]
#ig.plot(z, 'zachary_gt.eps', bbox=(0,0,300,200))
ig.plot(z, bbox=(0,0,350,250))

## Node roles --
 
We compute z(v) (normalized within module degree) and p(v) (participation coefficients) as defined in section 5.2 of the book. 

We identify 3 types of nodes

* provincial hubs
* peripheral nodes (non-hubs)
* ultra peripheral nodes (non-hubs)
    

In [ ]:
## internal degrees
in_deg_0 = z.subgraph_edges([e for e in z.es if z.vs['comm'][e.tuple[0]]==0 and z.vs['comm'][e.tuple[1]]==0],
                            delete_vertices=False).degree()
in_deg_1 = z.subgraph_edges([e for e in z.es if z.vs['comm'][e.tuple[0]]==1 and z.vs['comm'][e.tuple[1]]==1],
                            delete_vertices=False).degree()

## compute z (normalized within-module degree)
z.vs['in_deg'] = [in_deg_0[i] + in_deg_1[i] for i in range(z.vcount())]
mu = [np.mean(in_deg_0),np.mean(in_deg_1)]
sig = [np.std(in_deg_0),np.std(in_deg_1)]
z.vs['z'] = [(v['in_deg']-mu[v['comm']])/sig[v['comm']] for v in z.vs]

## computing p (participation coefficient)
z.vs['deg'] = z.degree()
z.vs['out_deg'] = [v['deg'] - v['in_deg'] for v in z.vs]
z.vs['p'] = [1-(v['in_deg']/v['deg'])**2-(v['out_deg']/v['deg'])**2 for v in z.vs]
D = pd.DataFrame(np.array([z.vs['z'],z.vs['p']]).transpose(),columns=['z','p']).sort_values(by='z',ascending=False)
D.head(8)


In [ ]:
## plotting w.r.t. z (z>2.5 are hubs, shown as squares)
## color positive values in dark, negative in white
## largest values for node 0 (instructor) and node 33 (president) and node 32
z.vs['color'] = 'black'
z.vs['shape'] = 'circle'
for v in z.vs:
    if v['z']<2.5: ## non-hub
        if v['p'] < .62 and v['p'] >= .05: ## peripheral
            v['color'] = 'dimgrey'
        if v['p'] < .05: ## ultra-peripheral
            v['color'] = 'gainsboro'
    if v['z']>=2.5 and v['p'] < .3: ## hubs (all provincial here)            
        v['color'] = 'silver'
        v['shape'] = 'square'
#ig.plot(z, 'zachary_roles_1.eps', bbox=(0,0,350,250))
ig.plot(z, bbox=(0,0,350,250))

In [ ]:
## Figure 5.3(b) -- comparing the roles
fig, ax = plt.subplots(figsize=(12,9))
ax.scatter(z.vs['p'],z.vs['z'],marker='o',s=75, color='k')

plt.plot([0, .5], [2.5, 2.5], color='k', linestyle='-', linewidth=2)
plt.plot([.05, .05], [-.5, 2.4], color='k', linestyle='-', linewidth=2)

ax.annotate('node 0', (z.vs['p'][0],z.vs['z'][0]-.05), xytext=(z.vs['p'][0]+.01,z.vs['z'][0]-.3), 
            fontsize=14,
            arrowprops = dict(  arrowstyle="-",connectionstyle="angle3,angleA=0,angleB=-90"))

ax.annotate('node 33', (z.vs['p'][33],z.vs['z'][33]-.05), xytext=(z.vs['p'][33]-.07,z.vs['z'][33]-.3), 
            fontsize=14,
            arrowprops = dict(  arrowstyle="-",connectionstyle="angle3,angleA=0,angleB=-90"))

ax.annotate('node 32', (z.vs['p'][32]-.005,z.vs['z'][32]), xytext=(z.vs['p'][32]-.07,z.vs['z'][32]), 
            fontsize=14,
            arrowprops = dict(  arrowstyle="-",connectionstyle="angle3,angleA=0,angleB=-90"))

ax.annotate('node 1', (z.vs['p'][1],z.vs['z'][1]-.05), xytext=(z.vs['p'][1]-.07,z.vs['z'][1]-.3), 
            fontsize=14,
            arrowprops = dict(  arrowstyle="-",connectionstyle="angle3,angleA=0,angleB=-90"))

ax.annotate('node 3', (z.vs['p'][3],z.vs['z'][3]-.05), xytext=(z.vs['p'][3]+.07,z.vs['z'][3]-.3), 
            fontsize=14,
            arrowprops = dict(  arrowstyle="-",connectionstyle="angle3,angleA=0,angleB=-90"))

ax.annotate('node 2', (z.vs['p'][2],z.vs['z'][2]-.05), xytext=(z.vs['p'][2]-.07,z.vs['z'][2]-.3), 
            fontsize=14,
            arrowprops = dict(  arrowstyle="-",connectionstyle="angle3,angleA=0,angleB=-90"))

ax.annotate('provincial hubs',(.3,3), fontsize=18)
ax.annotate('peripheral non-hubs',(.3,1.8), fontsize=18)
ax.annotate('ultra peripheral non-hubs',(.03,0.25),xytext=(.1,0.25), fontsize=18,
             arrowprops = dict( arrowstyle="->", connectionstyle="angle3,angleA=0,angleB=-90"))

plt.xlabel('participation coefficient (p)',fontsize=16)
plt.ylabel('normalized within module degree (z)',fontsize=16);
#plt.savefig('zachary_roles_2.eps')

### Checking the communities w.r.t. strong/weak definitions


In [ ]:
## strong criterion: internal degree is larger for each node
## only two nodes do not qualify
for i in range(z.vcount()):
    c = z.vs[i]['comm']
    n = [z.vs[v]['comm']==c for v in z.neighbors(i)]
    if sum(n)<=len(n)-sum(n):
        print('node',i,'has internal degree',sum(n),'external degree',len(n)-sum(n))

In [ ]:
## weak criterion: total internal degree > total external degree
## both communities satisfy this criterion
I = [0,0]
E = [0,0]
for i in range(z.vcount()):
    c = z.vs[i]['comm']
    n = [z.vs[v]['comm']==c for v in z.neighbors(i)]
    I[c] += sum(n)
    E[c] += len(n)-sum(n)
print('community 0 internal degree',I[0],'external degree',E[0])
print('community 1 internal degree',I[1],'external degree',E[1])


### Hierarchical clustering and dendrogram


In [ ]:
## Girvan-Newman algorithm
gn = z.community_edge_betweenness()
#ig.plot(gn,'zachary_dendrogram.eps',bbox=(0,0,300,300))
ig.plot(gn,bbox=(0,0,300,300))

In [ ]:
## compute modularity at each possible cut
q = []
for i in np.arange(z.vcount()):
    q.append(z.modularity(gn.as_clustering(n=i+1)))
plt.plot(np.arange(1,1+z.vcount()),q,'o-',color='black')
plt.xlabel('number of clusters',fontsize=14)
plt.ylabel('modularity',fontsize=14);
#plt.savefig('zachary_modularity.eps');

In [ ]:
## show result with 2 clusters -- only 1 node is misclassified
z.vs['gn'] = gn.as_clustering(n=2).membership
print('AMI:',AMI(z.vs['comm'],z.vs['gn']))  ## adjusted mutual information
print('q:',z.modularity(z.vs['gn']))        ## modularity

z.vs['size'] = 10
z.vs['name'] = [str(i) for i in range(z.vcount())]
z.vs['label'] = [str(i) for i in range(z.vcount())]
z.vs['label_size'] = 8
z.es['color'] = cls_edges
z.vs['comm'] = [0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1]
#z.vs['color'] = [cls[i] for i in z.vs['comm']]
z.vs['color'] = [cls[i] for i in z.vs['gn']]
#ig.plot(z, 'zachary_2.eps',bbox=(0,0,300,200))
ig.plot(z,bbox=(0,0,300,200))

In [ ]:
## show result with optimal modularity (5 clusters)
z.vs['label'] = gn.as_clustering(n=5).membership
print('AMI:',AMI(z.vs['comm'],z.vs['label']))
print('q:',z.modularity(z.vs['label']))
z.vs['color'] = [cls[i] for i in z.vs['comm']]
z.vs['size'] = 10
z.vs['label_size'] = 8
#ig.plot(z, 'zachary_5.eps',bbox=(0,0,300,200))
ig.plot(z,bbox=(0,0,300,200))

## ABCD graph with 100 nodes

This graph has 3 communities; with hierarchical clustering, we compare modularity and AMI for each possible cut.

Parameters: gamma=3, tau=2, degree range [5,15], comm size range [25,50], xi=.2.

In [ ]:
## read graph and communities
g = ig.Graph.Read_Ncol(datadir+'ABCD/abcd_100.dat',directed=False)
c = np.loadtxt(datadir+'ABCD/abcd_100_comms.dat',dtype='uint16',usecols=(1))
g.vs['comm'] = [c[int(x['name'])-1]-1 for x in g.vs]
gt = {k:(v-1) for k,v in enumerate(g.vs['comm'])}
## map between int(name) to key
n2k = {int(v):k for k,v in enumerate(g.vs['name'])}
g.vs['size'] = 7
g.es['color'] = cls_edges
g.vs['color'] = [cls[i] for i in g.vs['comm']]
ig.plot(g, bbox=(0,0,300,200))

### Girvan-Newman algorithm -- modularity and AMI for each cut

In [ ]:
q = []
a = []
gn = g.community_edge_betweenness()
for i in np.arange(g.vcount()):
    q.append(g.modularity(gn.as_clustering(n=i+1)))
    a.append(AMI(g.vs['comm'],gn.as_clustering(n=i+1).membership))
plt.plot(np.arange(1,1+g.vcount()),q,'.-',color='black',label='modularity')
plt.plot(np.arange(1,1+g.vcount()),a,'.-',color='grey',label='AMI')
plt.xlabel('number of clusters',fontsize=14)
plt.ylabel('modularity or AMI',fontsize=14)
plt.legend();
#plt.savefig('abcd_dendrogram.eps');

In [ ]:
D = pd.DataFrame(np.array([q,a]).transpose(),columns=['q','AMI'])
df = D.head()
df

In [ ]:
## AMI
g.vs['gn'] = gn.as_clustering(n=3).membership
print('AMI:',AMI(g.vs['comm'],g.vs['gn']))
print('q:',g.modularity(g.vs['gn']))

In [ ]:
## what would we get with 4 clusters?
## we see a few nodes get splitted from one community
g.vs['gn'] = gn.as_clustering(n=4).membership
cls = ['silver','dimgray','black','white']
g.vs['color'] = [cls[i] for i in g.vs['gn']]
#ig.plot(g, 'abcd_4.eps', bbox=(0,0,300,200))
ig.plot(g, bbox=(0,0,300,200))

In [ ]:
## those form a triangle
sg = g.subgraph([v for v in g.vs() if v['gn']==3])
ig.plot(sg, bbox=(0,0,100,100))

## ABCD with varying xi

Here we show a typical way to compare graph clustering using benchmark graphs. 

We pick some model, here ABCD, and we vary the noise parameter (0 <= xi <= 1). 

With ABCD, the larger xi is, the closer we are to a random Chung-Lu or configuration model graph (i.e. where only the degree distribution matters). 

For xi=0, we get pure communities (all edges are internal).

The code below can take a while to run; a pickle file with results is included in the Data directory;

To re-run from scratch, uncomment the cell below.


In [ ]:
## load L and train/val/test ids
L = pickle.load(open(datadir+"ABCD/abcd_study.pkl","rb"))
## store in dataframe and take averages
D = pd.DataFrame(L,columns=['algo','xi','AMI'])
X = D.groupby(by=['algo','xi']).mean()


In [ ]:
a = ['ECG','Louvain','Infomap','Label Prop.']
lt = ['-','--',':','-.','--',':']
cl = ['blue','green','purple','red','red','blue']
for i in range(len(a)):
    #plt.plot(X.loc[(a[i])].index,X.loc[(a[i])],lt[i],label=a[i],color=cl[i])
    plt.plot(X.loc[(a[i])].index,X.loc[(a[i])],lt[i],label=a[i],color='black')
plt.xlabel(r'ABCD noise ($\xi$)',fontsize=14)
plt.ylabel('AMI',fontsize=14)
plt.legend();
#plt.savefig('abcd_study.eps');

###  Look at standard deviations

In [ ]:
S = D.groupby(by=['algo','xi']).std()
a = ['ECG','Louvain','Infomap','Label Prop.']
#a = ['ECG','Louvain','Infomap','Label Prop.','Leiden','CNM']
lt = ['-','--',':','-.','--',':']
cl = ['blue','green','purple','red','red','blue']
for i in range(len(a)):
    #plt.plot(S.loc[(a[i])].index,S.loc[(a[i])],lt[i],label=a[i],color=cl[i])
    plt.plot(S.loc[(a[i])].index,S.loc[(a[i])],lt[i],label=a[i],color='black')
plt.xlabel(r'ABCD noise ($\xi$)',fontsize=14)
plt.ylabel('Standard Deviation (AMI)',fontsize=14)
plt.legend();
#plt.savefig('abcd_study_stdv.eps');

### Compare stability 

This study is similar to the previous one, but we compare successive partitions for each algorithm instead of comparing with the ground truth.

The code below can take a while to run; a pickle file with results is included in the Data directory;

To re-run from scratch, uncomment the cell below.


In [ ]:
## load L and train/val/test ids
Ls = pickle.load(open(datadir+"ABCD/abcd_study_stability.pkl","rb"))

## store in dataframe and take averages
D = pd.DataFrame(Ls,columns=['algo','xi','AMI'])
X = D.groupby(by=['algo','xi']).mean()

In [ ]:
a = ['ECG','Louvain','Infomap','Label Prop.']
lt = ['-','--',':','-.']
for i in range(len(a)):
    plt.plot(X.loc[(a[i])].index,X.loc[(a[i])],lt[i],label=a[i],color='black')
plt.xlabel(r'ABCD noise ($\xi$)',fontsize=14)
plt.ylabel('AMI between successive runs',fontsize=14)
plt.legend();
#plt.savefig('abcd_study_stability.eps');

## Modularity, resolution limit and rings of cliques

In [ ]:
## n cliques of size s
def ringOfCliques(n,s):
    roc = ig.Graph.Erdos_Renyi(n=n*s,p=0)
    ## cliques
    for i in range(n):
        for j in np.arange(s*i,s*(i+1)):
            for k in np.arange(j+1,s*(i+1)):
                roc.add_edge(j,k)
    ## ring
    for i in range(n):
        if i>0:
            roc.add_edge(s*i-1,s*i)
        else:
            roc.add_edge(n*s-1,0)
    roc.vs['size'] = 8
    roc.vs['color'] = cls[2]
    roc.es['color'] = cls_edges
    return roc

## Ex: 10 3-cliques
roc = ringOfCliques(10,3)
#ig.plot(roc,'ring_3.eps',bbox=(0,0,300,300))     
ig.plot(roc,bbox=(0,0,300,300))        

In [ ]:
## Compare number of cliques and number of clusters found
L = []
s = 3
for n in np.arange(3,50,3):
    roc = ringOfCliques(n,s)
    ml = np.max(roc.community_multilevel().membership)+1
    ec = np.max(roc.community_ecg().membership)+1
    cnm = np.max(roc.community_fastgreedy().as_clustering().membership)+1
    L.append([n,ml,ec,cnm])
D = pd.DataFrame(L,columns=['n','Louvain','ECG','CNM'])
plt.figure(figsize=(8,6))
plt.plot(D['n'],D['Louvain'],'--o',color='black',label='Louvain')
plt.plot(D['n'],D['ECG'],'-o',color='black',label='ECG')
plt.plot(D['n'],D['CNM'],':o',color='black',label='CNM')

plt.xlabel('number of '+str(s)+'-cliques',fontsize=14)
plt.ylabel('number of clusters found',fontsize=14)
plt.legend(fontsize=14);
#plt.savefig('rings.eps');

In [ ]:
## Louvain communities with 10 3-cliques
roc = ringOfCliques(n=10,s=3)
roc.vs['ml'] = roc.community_multilevel().membership
roc.vs['color'] = [cls[x%3] for x in roc.vs['ml']]
#ig.plot(roc,'ring_3_q.eps', bbox=(0,0,300,300))
ig.plot(roc,bbox=(0,0,300,300))

In [ ]:
## ECG weights in this case: all 30 clique edges have max score
roc.es['W'] = roc.community_ecg().W
Counter(roc.es['W'])

# Ego nets and more

* we consider the airport graph we already saw
* we consider a simple, undirected version (no loops, directions or edge weights)
* we compare ego-nets (1 and 2-hops subgraphs from a given node) with clusters obtained via graph clustering

In [ ]:
## read edges and build simple undirected graph
D = pd.read_csv(datadir+'Airports/connections.csv')
g = ig.Graph.TupleList([tuple(x) for x in D.values], directed=True, edge_attrs=['weight'])
#df = D.head()
g = g.as_undirected()
g = g.simplify()

In [ ]:
## read vertex attributes and add to graph
A = pd.read_csv(datadir+'Airports/airports_loc.csv')
lookup = {k:v for v,k in enumerate(A['airport'])}
l = [lookup[x] for x in g.vs()['name']]
g.vs()['layout'] = [(A['lon'][i],A['lat'][i]) for i in l]
g.vs()['state'] = [A['state'][i] for i in l]
g.vs()['city'] = [A['city'][i] for i in l]
## add a few more attributes for visualization
g.vs()['size'] = 6
g.vs()['color'] = cls[0]
g.es()['color'] = cls_edges
df = A.head()

In [ ]:
## pick a vertex
v = 207
print(g.degree()[v],g.vs[v])
g.vs[v]['color'] = 'black'

In [ ]:
## show its ego-net
sg = g.subgraph([i for i in g.neighborhood(v,order=1)])
print(sg.vcount(),'nodes')
#ig.plot(sg,'airport_ego_1.eps',bbox=(0,0,300,300))
ig.plot(sg,bbox=(0,0,300,300))

In [ ]:
## show its 2-hops ego-net, this is already quite large!
sg = g.subgraph([i for i in g.neighborhood(v,order=2)])
sg.vs()['core'] = sg.coreness()
sg.delete_vertices([v for v in sg.vs if v['core']<2])
print(sg.vcount(),'nodes')
#ig.plot(sg,'airport_ego_2.eps',bbox=(0,0,300,300))
ig.plot(sg,bbox=(0,0,300,300))

In [ ]:
## apply clustering, show cluster containing the selected vertex
## recall that we ignore edge weights
ec = g.community_ecg(ens_size=32)
g.es['W'] = ec.W
m = ec.membership[v]
sg = g.subgraph([i for i in range(g.vcount()) if ec.membership[i]==m])
sg.vs()['core'] = sg.coreness()
## display the 2-core
sg.delete_vertices([v for v in sg.vs if v['core']<2])
print(sg.vcount(),'nodes')
#ig.plot(sg,'airport_ecg.eps',bbox=(0,0,300,300))
ig.plot(sg,bbox=(0,0,300,300))

In [ ]:
## filter edges w.r.t. ECG votes (weights)
## you can adjust the threshold to get different zooming
thresh = .9
tmp = sg.subgraph_edges([e for e in sg.es if e['W'] > thresh])
n = [i for i in range(tmp.vcount()) if tmp.vs[i]['color']=='black'][0]
tmp.vs['cl'] = tmp.clusters().membership
cl = tmp.vs[n]['cl']
ssg = tmp.subgraph([i for i in tmp.vs if i['cl']==cl])
ssg.vs()['core'] = ssg.coreness()
ssg.delete_vertices([v for v in ssg.vs if v['core']<2])
print(ssg.vcount(),'nodes')
#ig.plot(ssg,'airport_ecg_focus.eps',bbox=(0,0,300,300))
ig.plot(ssg,bbox=(0,0,300,300))

In [ ]:
## states in the above subgraph
Counter(ssg.vs['state'])

# ABCD Properties

We show ABCD graphs with different xi (noise) parameters;

This is for illustration purpose only:

* notice the density of edges between communities as xi increases.
* most runs should yield 3 communities, but this can vary when we re-run ABCD samplers

In [ ]:
## generate new degree and community size values
cmd = 'julia '+abcd_path+'deg_sampler.jl deg.dat 2.5 5 15 100 1000'
os.system(cmd)
cmd = 'julia '+abcd_path+'com_sampler.jl cs.dat 1.5 30 50 100 1000'
os.system(cmd);


In [ ]:
## just for visualization -- push the layout apart given 3 communities
def push_layout(d=0):
    if np.max(g.vs['comm'])>2:
        return -1
    ly = g.layout()
    g.vs['ly'] = ly
    x = [0,0,0]
    y = [0,0,0]
    for v in g.vs:
        c = v['comm']
        x[c] += v['ly'][0]
        y[c] += v['ly'][1]
    delta = [-d,0,d]
    dx = [delta[i] for i in np.argsort(x)]
    dy = [delta[i] for i in np.argsort(y)]
    for v in g.vs:
        c = v['comm']
        v['ly'][0] += dx[c]
        v['ly'][1] += dy[c]
    return g.vs['ly']

In [ ]:
## viz: ABCD with very strong communities
## results will vary, but we see 3 communities in most runs.
xi = 0.05
cmd = 'julia '+abcd_path+'graph_sampler.jl net.dat comm.dat deg.dat cs.dat '\
        +str(xi)+' false false'
os.system(cmd)
## compute AMI for various clustering algorithms
g = ig.Graph.Read_Ncol('net.dat',directed=False)
c = np.loadtxt('comm.dat',dtype='uint16',usecols=(1))
g.vs['comm'] = [c[int(x['name'])-1]-1 for x in g.vs]
g.vs['color'] = [cls[i] for i in g.vs['comm']]
g.vs['size'] = 5
g.es['color'] = 'lightgrey'
ly = push_layout(0)
ig.plot(g, layout=ly, bbox=(0,0,300,300))

In [ ]:
## viz: ABCD with strong communities
xi = 0.15
cmd = 'julia '+abcd_path+'graph_sampler.jl net.dat comm.dat deg.dat cs.dat '\
        +str(xi)+' false false'
os.system(cmd)
## compute AMI for various clustering algorithms
g = ig.Graph.Read_Ncol('net.dat',directed=False)
c = np.loadtxt('comm.dat',dtype='uint16',usecols=(1))
g.vs['comm'] = [c[int(x['name'])-1]-1 for x in g.vs]
g.vs['color'] = [cls[i] for i in g.vs['comm']]
g.vs['size'] = 5
g.es['color'] = 'lightgrey'
ly = push_layout(1)
ig.plot(g, layout=ly, bbox=(0,0,300,300))


In [ ]:
## viz: ABCD with weak communities
xi = 0.33
cmd = 'julia '+abcd_path+'graph_sampler.jl net.dat comm.dat deg.dat cs.dat '\
        +str(xi)+' false false'
os.system(cmd)
## compute AMI for various clustering algorithms
g = ig.Graph.Read_Ncol('net.dat',directed=False)
c = np.loadtxt('comm.dat',dtype='uint16',usecols=(1))
g.vs['comm'] = [c[int(x['name'])-1]-1 for x in g.vs]
g.vs['color'] = [cls[i] for i in g.vs['comm']]
g.vs['size'] = 5
g.es['color'] = 'lightgrey'
ly = push_layout(6)
ig.plot(g, layout=ly, bbox=(0,0,300,300))

In [ ]:
## viz: ABCD with very weak communities
xi = 0.5
cmd = 'julia '+abcd_path+'graph_sampler.jl net.dat comm.dat deg.dat cs.dat '\
        +str(xi)+' false false'
os.system(cmd)
## compute AMI for various clustering algorithms
g = ig.Graph.Read_Ncol('net.dat',directed=False)
c = np.loadtxt('comm.dat',dtype='uint16',usecols=(1))
g.vs['comm'] = [c[int(x['name'])-1]-1 for x in g.vs]
g.vs['color'] = [cls[i] for i in g.vs['comm']]
g.vs['size'] = 5
g.es['color'] = 'lightgrey'
ly = push_layout(6)
ig.plot(g, layout=ly, bbox=(0,0,300,300))

## Measures

* We illustrate the importance of using proper adjusted measures when comparing partitions
* We generate some ABCD graph and compare ground truth with random partitions of different sizes

In [ ]:
## generate new degree and community size values
cmd = 'julia '+abcd_path+'deg_sampler.jl deg.dat 2.5 5 50 1000 1000'
os.system(cmd)
cmd = 'julia '+abcd_path+'com_sampler.jl cs.dat 1.5 75 150 1000 1000'
os.system(cmd)
xi = .1
cmd = 'julia '+abcd_path+'graph_sampler.jl net.dat comm.dat deg.dat cs.dat '\
        +str(xi)+' false false'
os.system(cmd)
## compute AMI for various clustering algorithms
g = ig.Graph.Read_Ncol('net.dat',directed=False)
c = np.loadtxt('comm.dat',dtype='uint16',usecols=(1))
gt = [c[int(x['name'])-1]-1 for x in g.vs]
print(np.max(gt)+1)

In [ ]:
## RAND Index: given two clusterings u and v
def RI(u,v):
    ## build sets from A and B
    a = np.max(u)+1
    b = np.max(v)+1
    n = len(u)
    if n != len(v):
        exit -1
    A = [set() for i in range(a)]
    B = [set() for i in range(b)]
    for i in range(n):
        A[u[i]].add(i)
        B[v[i]].add(i)   
    ## RAND index step by step
    R = 0
    for i in range(a):
        for j in range(b):
            s = len(A[i].intersection(B[j]))
            if s>1:
                R += s*(s-1)/2
    R *= 2
    for i in range(a):
        s = len(A[i])
        if s>1:
            R -= s*(s-1)/2
    for i in range(b):
        s = len(B[i])
        if s>1:
            R -= s*(s-1)/2
    R += n*(n-1)/2
    R /= n*(n-1)/2
    return R
    

In [ ]:
## generate random clusterings and compute various measures w.r.t. ground truth
from sklearn.metrics import mutual_info_score as MI
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import normalized_mutual_info_score as NMI
L = []
n = g.vcount()
ar = np.arange(2,21)
for s in ar:
    for i in range(100):
        r = np.random.choice(s, size=n)
        L.append([s,MI(gt,r),NMI(gt,r),AMI(gt,r),RI(gt,r),ARI(gt,r)])

In [ ]:
## Mutual information (MI), normalized MI (NMI) and adjusted MI (AMI)

D = pd.DataFrame(L,columns=['size','MI','NMI','AMI','RI','ARI'])
R = D.groupby(by='size').mean()

plt.plot(ar,R['MI'],':',color='black',label='MI')
plt.plot(ar,R['NMI'],'--',color='black',label='NMI')
plt.plot(ar,R['AMI'],'-',color='black',label='AMI')
plt.xlabel('number of random clusters',fontsize=14)
plt.legend();
#plt.savefig('MI.eps');


In [ ]:
## RAND index (RI) and adjusted (ARI)
plt.plot(ar,R['RI'],':',color='black',label='RI')
plt.plot(ar,R['ARI'],'-',color='black',label='ARI')
plt.xlabel('number of random clusters',fontsize=14)
plt.legend();
#plt.savefig('RI.eps');